In [32]:
import numpy as np
import pandas as pd

from wordcloud import WordCloud, STOPWORDS
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,auc,roc_curve,confusion_matrix,classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier

import os
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:

print(train.shape, test.shape)

(20800, 5) (5200, 4)


In [6]:
print(train.isnull().sum())
print("-"*20)
print(test.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
--------------------
id          0
title     122
author    503
text        7
dtype: int64


In [7]:
train = train.fillna(' ')
test = test.fillna(' ')

In [8]:
test['total'] = test['title'] + ' ' + test['author'] + ' ' + test['text']
train['total'] = train['title'] + ' ' + train['author'] + ' ' + train['text']

In [9]:
train.head(3)

,id,title,author,text,label,total
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...


In [10]:
test.head(3)

,id,title,author,text,total
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...","Specter of Trump Loosens Tongues, if Not Purse..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,#NoDAPL: Native American Leaders Vow to Stay A...


In [11]:
train[train['label']==1].total

,total
0,House Dem Aide: We Didn’t Even See Comey’s Let...
2,Why the Truth Might Get You Fired Consortiumne...
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...
...,...
20788,Maine’s Gov. LePage Threatens To ‘Investigate’...
20791,Lawyer Who Kept Hillary Campaign Chief Out of ...
20793,Idiot Who Destroyed Trump Hollywood Star Gets ...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:

from tqdm import tqdm
real_words = ''
fake_words = ''
#iterating through each row in train where label is 1, to get only real words
for val in tqdm(train[train['label']==1].total):

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    real_words += " ".join(tokens)+" "

#iterating through each row in train where label is 1, to get only real words
for val in tqdm(train[train['label']==0].total):

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    fake_words += " ".join(tokens)+" "

100%|██████████| 10387/10387 [00:09<00:00, 1050.48it/s]


In [16]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

for index, row in tqdm(train.iterrows()):
    filter_sentence = ''
    sentence = row['total']

    #removing all one word characters and whitespaces using regular expressions
    sentence = re.sub(r'[^\w\s]', '', sentence)

    # Tokenization, spliting the sentence into tokens
    words = nltk.word_tokenize(sentence)

    # Stopwords removal
    words = [w for w in words if not w in stop_words]

    #lemmatization, getting tokens into their base form
    for words in words:
        filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()

    train.loc[index, 'total'] = filter_sentence

[nltk_data] Downloading package wordnet to /root/nltk_data...
20800it [02:56, 118.04it/s]


In [18]:
for index, row in tqdm(test.iterrows()):
    filter_sentence = ''
    sentence = row['total']

    # Cleaning the sentence with regex
    sentence = re.sub(r'[^\w\s]', '', sentence)

    # Tokenization
    words = nltk.word_tokenize(sentence)

    # Stopwords removal
    words = [w for w in words if not w in stop_words]

    for words in words:
        filter_sentence = filter_sentence  + ' ' + str(lemmatizer.lemmatize(words)).lower()

    test.loc[index, 'total'] = filter_sentence

5200it [00:50, 102.44it/s]


In [19]:
train = train[['total', 'label']]

In [20]:
train.head()

,total,label
0,house dem aide we didnt even see comeys lette...,1
1,flynn hillary clinton big woman campus breitb...,0
2,why truth might get you fired consortiumnewsc...,1
3,15 civilians killed in single us airstrike ha...,1
4,iranian woman jailed fictional unpublished st...,1


In [21]:

X_train = train['total']
Y_train = train['label']

In [22]:

test = test['total']

In [23]:

test.head()

,total
0,specter trump loosens tongues not purse strin...
1,russian warship ready strike terrorist near a...
2,nodapl native american leaders vow stay all w...
3,tim tebow will attempt another comeback this ...
4,keiser report meme wars e995 truth broadcast ...


In [24]:
#TFIDF vectorizer , transformer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(X_train)
tfidf = TfidfTransformer(norm = "l2")
tf_idf_matrix = tfidf.fit_transform(tf_idf_matrix)

In [25]:
tf_idf_matrix.shape

(20800, 210578)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y_train, random_state=0)

In [30]:
res=[]
def model_check(name, model):
  model= model.fit(X_train, y_train)
  y_predict= model.predict(X_test)

  model_accuracy=accuracy_score(y_test,y_predict)

  data ={'Model_Name': name,
         'accuracy': model_accuracy}

  res.append(data)

In [33]:
model_check('Desiciontree', DecisionTreeClassifier())

In [34]:
model_check('Randomforest', RandomForestClassifier())

In [35]:
model_check('GradientBoostingClassifier',  GradientBoostingClassifier())

In [36]:
model_check('AdaBoostClassifier',  AdaBoostClassifier())

In [37]:
model_check('ExtraTreesClassifier',  ExtraTreesClassifier())

In [38]:
model_df = pd.DataFrame(res)
model_df

,Model_Name,accuracy
0,Desiciontree,0.969423
1,Randomforest,0.946346
2,GradientBoostingClassifier,0.970000
3,AdaBoostClassifier,0.968654
4,ExtraTreesClassifier,0.932308


In [ ]:
# for this dataset,  D	GradientBoostingClassifier gives better result than others

In [42]:
X_train = train['total']
Y_train = train['label']

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', GradientBoostingClassifier()),
])

In [44]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier())])

In [45]:
test.head()

,total
0,specter trump loosens tongues not purse strin...
1,russian warship ready strike terrorist near a...
2,nodapl native american leaders vow stay all w...
3,tim tebow will attempt another comeback this ...
4,keiser report meme wars e995 truth broadcast ...


In [46]:
pred_label = pipeline.predict(test)

In [48]:
pred_label

array([0, 1, 1, ..., 0, 1, 0])

In [49]:
pred_label.shape

(5200,)